In [ ]:
rawRDD = sc.textFile('data/covtype.data.gz')

In [ ]:
column_names = (
["Elevation",
"Aspect",
"Slope",
"Horizontal_Distance_To_Hydrology",
"Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am",
"Hillshade_Noon",
"Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"]
    + ['WE{}'.format(i) for i in range(4)]
    + ['ST{}'.format(i) for i in range(40)]
    + ['Cover'])

In [ ]:
from pyspark.sql import Row

In [ ]:
cover = spark.createDataFrame(
    rawRDD
     .map(lambda x: [float(i) for i in x.split(',')])
     .map(lambda x: Row(**{col:v for col, v in zip(column_names, x)})))

In [ ]:
cover.select(cover['elevation']).show(5)

In [ ]:
cover.select(cover['elevation'] / 10).show(5)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feature_maker = VectorAssembler(inputCols=column_names[:-1], outputCol='features')

In [ ]:
data = feature_maker.transform(cover)

In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
model = LogisticRegression(labelCol='Cover').fit(train)

In [ ]:
model.hasSummary

In [ ]:
print model

In [ ]:
feature_maker = VectorAssembler(inputCols=column_names[:10], outputCol='features')

In [ ]:
data = feature_maker.transform(cover)
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
binaryTrain = train.select('features', (train['Cover'] - 1).alias('Cover')).filter((train['Cover'] == 1) | (train['Cover'] == 0))

In [ ]:
from pyspark.ml.regression import GeneralizedLinearRegression

In [ ]:
binaryModel = GeneralizedLinearRegression(link='logit', family='binomial', labelCol='Cover', maxIter=10, regParam=0.3).fit(binaryTrain)

In [ ]:
binaryModel.summary